# Watershed for Parcellation
1. Use watershed to parcellate activation maps 
2. generate one map for each region 
3. find the local maxima of each region and mni coordinate 

In [ ]:
import numpy as np
import pandas as pd
import os
import json
import time

from algo import *
from skimage import segmentation
from scipy import ndimage
from collections import Counter
from nilearn import image, plotting, datasets, surface

## Relavant Functions

In [ ]:
def is_local_maximum(image, labels=None, footprint=None):
    """Return a boolean array of points that are local maxima
 
    Parameters
    ----------
    image: ndarray (2-D, 3-D, ...)
        intensity image
        
    labels: ndarray, optional 
        find maxima only within labels. Zero is reserved for background.
    
    footprint: ndarray of bools, optional
        binary mask indicating the neighborhood to be examined
        `footprint` must be a matrix with odd dimensions, the center is taken 
        to be the point in question.
    Returns
    -------
    result: ndarray of bools
        mask that is True for pixels that are local maxima of `image`
    Notes
    -----
    This function is copied from watershed module in CellProfiler.
    This module implements a watershed algorithm that apportions pixels into
    marked basins. The algorithm uses a priority queue to hold the pixels
    with the metric for the priority queue being pixel value, then the time
    of entry into the queue - this settles ties in favor of the closest marker.
    Some ideas taken from
    Soille, "Automated Basin Delineation from Digital Elevation Models Using
    Mathematical Morphology", Signal Processing 20 (1990) 171-182.
    The most important insight in the paper is that entry time onto the queue
    solves two problems: a pixel should be assigned to the neighbor with the
    largest gradient or, if there is no gradient, pixels on a plateau should
    be split between markers on opposite sides.
    Originally part of CellProfiler, code licensed under both GPL and BSD licenses.
    Website: http://www.cellprofiler.org
    Copyright (c) 2003-2009 Massachusetts Institute of Technology
    Copyright (c) 2009-2011 Broad Institute
    All rights reserved.
    Original author: Lee Kamentsky
    Examples
    --------
    >>> image = np.zeros((4, 4))
    >>> image[1, 2] = 2
    >>> image[3, 3] = 1
    >>> image
    array([[ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  2.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  1.]])
    >>> is_local_maximum(image)
    array([[ True, False, False, False],
           [ True, False,  True, False],
           [ True, False, False, False],
           [ True,  True, False,  True]], dtype='bool')
    >>> image = np.arange(16).reshape((4, 4))
    >>> labels = np.array([[1, 2], [3, 4]])
    >>> labels = np.repeat(np.repeat(labels, 2, axis=0), 2, axis=1)
    >>> labels
    array([[1, 1, 2, 2],
           [1, 1, 2, 2],
           [3, 3, 4, 4],
           [3, 3, 4, 4]])
    >>> image
    array([[ 0,  1,  2,  3],
           [ 4,  5,  6,  7],
           [ 8,  9, 10, 11],
           [12, 13, 14, 15]])
    >>> is_local_maximum(image, labels=labels)
    array([[False, False, False, False],
           [False,  True, False,  True],
           [False, False, False, False],
           [False,  True, False,  True]], dtype='bool')
    """
    if labels is None:
        labels = np.ones(image.shape, dtype=np.uint8)
    if footprint is None:
        footprint = np.ones([3] * image.ndim, dtype=np.uint8)
    assert((np.all(footprint.shape) & 1) == 1)
    footprint = (footprint != 0)
    footprint_extent = (np.array(footprint.shape)-1) // 2
    if np.all(footprint_extent == 0):
        return labels > 0
    result = (labels > 0).copy()
    #
    # Create a labels matrix with zeros at the borders that might be
    # hit by the footprint.
    #
    big_labels = np.zeros(np.array(labels.shape) + footprint_extent*2,
                          labels.dtype)
    big_labels[[slice(fe,-fe) for fe in footprint_extent]] = labels
    #
    # Find the relative indexes of each footprint element
    #
    image_strides = np.array(image.strides) // image.dtype.itemsize
    big_strides = np.array(big_labels.strides) // big_labels.dtype.itemsize
    result_strides = np.array(result.strides) // result.dtype.itemsize
    footprint_offsets = np.mgrid[[slice(-fe,fe+1) for fe in footprint_extent]]
    
    fp_image_offsets = np.sum(image_strides[:, np.newaxis] *
                              footprint_offsets[:, footprint], 0)
    fp_big_offsets = np.sum(big_strides[:, np.newaxis] *
                            footprint_offsets[:, footprint], 0)
    #
    # Get the index of each labeled pixel in the image and big_labels arrays
    #
    indexes = np.mgrid[[slice(0,x) for x in labels.shape]][:, labels > 0]
    image_indexes = np.sum(image_strides[:, np.newaxis] * indexes, 0)
    big_indexes = np.sum(big_strides[:, np.newaxis] * 
                         (indexes + footprint_extent[:, np.newaxis]), 0)
    result_indexes = np.sum(result_strides[:, np.newaxis] * indexes, 0)
    #
    # Now operate on the raveled images
    #
    big_labels_raveled = big_labels.ravel()
    image_raveled = image.ravel()
    result_raveled = result.ravel()
    #
    # A hit is a hit if the label at the offset matches the label at the pixel
    # and if the intensity at the pixel is greater or equal to the intensity
    # at the offset.
    #
    for fp_image_offset, fp_big_offset in zip(fp_image_offsets, fp_big_offsets):
        same_label = (big_labels_raveled[big_indexes + fp_big_offset] ==
                      big_labels_raveled[big_indexes])
        less_than = (image_raveled[image_indexes[same_label]] <
                     image_raveled[image_indexes[same_label]+ fp_image_offset])
        result_raveled[result_indexes[same_label][less_than]] = False
        
    return result

In [ ]:
def watershed(data, sigma=1, thresh=0, seeds=None):
    """The implementation of watershed algorithm."""
    
    thresh = thresh > 0 and thresh
    if thresh == 0:
        mask = data > thresh 
    else:
        mask = data >= thresh
    data = ndimage.gaussian_filter(data, sigma)
    if seeds is None:
        # using unmasked data to get local maximum
        seeds = is_local_maximum(data)
    # mask out those smaller than threshold
    seeds[~mask] = 0

    se = ndimage.generate_binary_structure(3, 3)
    markers = ndimage.label(seeds, se)[0]
    
    seg_input =  mask

    result = segmentation.watershed(seg_input, markers, mask=mask)
    return result

In [ ]:
def to_mni(data_array):
    """
    convert array(beta) to mni space
    specific for the analysis

    Parameter
    ---------
    data_array: ndarray
        1D array, contraining beta value

    Return
    ------
    mni_data: Nifti1Image
        NiftiImage, with space and afine specific for this analysis
    """
    # acquire sample parameters
    sample_path = '/Users/anmin/Documents/attsig_data/data/beta_0001.nii'
    sample_nii = nib.load(sample_path)
    ori_shape = np.array(sample_nii.get_data()).shape
    affine = sample_nii.affine.copy()
    hdr = sample_nii.header.copy()

    del sample_nii

    mni_data = nib.Nifti1Image(data_array.reshape(ori_shape),affine,hdr)

    return mni_data

## Load Data

In [ ]:
root_path = '/Users/anmin/Documents/attsig_data/'

fa = image.get_data(os.path.join(root_path,
                            'nii_file',
                            'fa_fdr_corrected.nii.gz'))
    
sa = image.get_data(os.path.join(root_path,
                            'nii_file',
                            'sa_fdr_corrected.nii.gz'))

mask_nan = image.get_data(os.path.join(root_path, 'nii_file',
                                        'mask_nan.nii.gz'))
fa_mask = np.load(os.path.join(root_path, 
                                'data', 'statistical_masks',
                                'fa_fdr.npy'))
sa_mask = np.load(os.path.join(root_path, 
                                'data', 'statistical_masks',
                                'sa_fdr.npy'))

In [ ]:
save_path = os.path.join(root_path,
                        'nii_file', 'cluster_map_small_version')
# the original cluster parcelation is in the file cluster map
# in the file small version, the cluster for SA is smaller 

In [ ]:
# read HO map mapper 
json_path = os.path.join(root_path,
                        'labelmapper.json')

In [ ]:
f = open(json_path)
map_lst = json.load(f)
cat_num = 49

mapper = {}
for i in range(cat_num):
    mapper[i] = map_lst[i][2] # here the mapper is changed to abreviation of the region 
                            # instead of the full name of the region

## FA 

### Watershed Parcellation 

In [ ]:
mask = np.isnan(fa)

# here the assumaptuon is the sign of weight is irrelevant, but rather
# the intensity
fa = abs(fa)

In [ ]:
data = fa
thresh = 0
mask = data > thresh 
sigma = 0.1
data = ndimage.gaussian_filter(data, sigma)
seeds = is_local_maximum(data)
seeds[~mask] = 0
se = ndimage.generate_binary_structure(3, 3)
markers = ndimage.label(seeds, se)[0]
seg_input = mask

parcel_map = segmentation.watershed(seg_input, markers, mask=mask)

In [ ]:
def find_loi(p_map, mask, v_num):
    """
    find the cluster label given minimum cluster size

    Parameters
    ----------
    p_map : ndarray 3D 
        activation map
    mask : ndarray 3D
        relavant voxels, same shape with p_map
    v_num : int
        minimun size of a cluster 
    
    Retruns
    -------
    selected_cluster : list of tuple
        (label, size)
    """
    counter = Counter(p_map[mask])

    selected_loi = []
    for key, val in counter.items():
        if val > v_num:
            selected_loi.append((key, val))
    
    return selected_loi 

In [ ]:
def mask_cat(mask, parcel_map):
    p_map = parcel_map.flatten()
    for i in range(mask.size):
        if not mask[i]:
            p_map[i] = np.nan 
    return p_map

In [ ]:
selected_loi = find_loi(parcel_map, mask, 10) # the threshold is set to 10 instead of 100

In [ ]:
len(selected_loi)

In [ ]:
parcel_map = parcel_map.astype('float')

### Generate nii file for each label 

In [ ]:
for label, num in selected_loi: 
    if label == 0:
        continue
    mask_l = parcel_map.flatten() == label

    mni_vec = mask_cat(mask_l, parcel_map.flatten())
    nif_fa = to_mni(mni_vec)

    nib.save(nif_fa, 
         os.path.join(save_path, 'fa',
                    f'{label}_{num}_fa.nii.gz'))

### Generate nii file for every label 

In [ ]:
# gen selectec mask 
mni_vec = []
p_map = parcel_map.flatten()
selected_cat = [i for i, j in selected_loi]

for val in p_map:
    if val in selected_cat:
        mni_vec.append(val)
    else:
        mni_vec.append(np.nan)

        
nif_fa = to_mni(np.array(mni_vec))
nib.save(nif_fa, 
         os.path.join(save_path, 'fa',
                    f'full_fa.nii.gz'))

### Cluster Evaluation

In [ ]:
dataset_ho = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
ho_map = image.get_data(dataset_ho['maps']) 

In [ ]:
niimg = datasets.load_mni152_template()
cortex_mask = ~(ho_map == 0).flatten() # ristrict maximum value in cortex 
table_dic = {}
total_num = len(selected_loi)

for label, num in selected_loi:
    start = time.time()
    if label == 0:
        continue    
    table_dic[label] = {}
    mask = image.get_data(os.path.join(save_path, 'fa',
                        f'{label}_{num}_fa.nii.gz'))
    mask = mask == label
    model_shape = mask.shape
    mask = mask.flatten()
    mask_with_cortex = mask * cortex_mask
    fa_arr = fa.flatten()

    # get beta values specific to label
    arr_to_max = []
    for i, val in enumerate(mask_with_cortex):
        if val:
            arr_to_max.append(fa_arr[i])
        else:
            arr_to_max.append(np.nan)

    arr_to_max = np.array(arr_to_max).reshape(model_shape)
    arr_to_max = np.nan_to_num(arr_to_max)

    # get cluster size
    table_dic[label]['voxel_size'] = num

    # get matrix coordinates
    x, y, z = np.unravel_index(np.argmax(arr_to_max), arr_to_max.shape)
    table_dic[label]['voxel_location'] = (x, y, z)

    # get mni coordinates
    table_dic[label]['mni_coordinates'] = image.coord_transform(x, y, z, 
                                                            niimg.affine)
    # get the right hemisphere or left hemisphere   
    hem = 'left ' if table_dic[label]['mni_coordinates'][0] < 0 else 'right '
 
    # get the label of H-O atlas 
    table_dic[label]['HO_categor_by_peak'] = ho_map[x, y, z]

    # get name of the cluster 
    names = mapper[table_dic[label]['HO_categor_by_peak']]
    table_dic[label]['names_by_peak'] = hem + names

    # sanity check, if the peak is in the left hemisphere
    if hem == 'left ':
        table_dic[label]['HO_categor_by_peak'] = -1 * ho_map[x, y, z]

    # get cluster name by covered region 
    label_count_arr = ho_map[mask.reshape(model_shape)]
    count_dic = Counter(label_count_arr)
    total = sum(count_dic.values())

    cat_percentile = {}
    for k in count_dic.keys():
        cat_percentile[k] = count_dic[k] / total 
    cat_percentile = sorted(cat_percentile.items(), key=lambda x: -x[1])

    table_dic[label]['percentile_distribution'] = cat_percentile
    table_dic[label]['HO_categor_by_area'] = cat_percentile[0][0]
    table_dic[label]['names_by_area'] = hem + mapper[cat_percentile[0][0]]

    # get intensity 
    table_dic[label]['z-score'] = (np.mean(fa_arr[mask]), np.std(fa_arr[mask]))

    # printout
    end = time.time()
    print('--------------------------------------------')
    print(f'{label} finished, time used: {end - start:.2f} seconds, ')
    total_num -= 1
    print(f'{total_num} left')

In [ ]:
df_fa = pd.DataFrame(table_dic).transpose()

df_fa.to_csv(os.path.join(root_path,
                        'cluster_table_small_version',
                        'fa_attributes.csv'))

#### Modification of Clusters
1. Directly read file
2. the standard of cluster label is 'HO_categor_by_peak'
3. merge clusters with the same label to the finall nii file 

In [ ]:
df_fa = pd.read_csv(os.path.join(root_path, 'cluster_table_small_version',
                                'fa_attributes.csv'))

In [ ]:
cats = np.unique(df_fa['HO_categor_by_peak'])
fa_arr = fa.flatten()

total_num = len(cats)

for cat in cats:
    if cat == 0: # drop background
        continue

    start = time.time()
    base_arr = np.zeros((fa_arr.size))
    df_temp = df_fa[df_fa['HO_categor_by_peak'] == cat ]
    for _, row in df_temp.iterrows():
        label = row['Unnamed: 0']
        num = row['voxel_size']
        mask = image.get_data(os.path.join(save_path, 'fa',
                        f'{label}_{num}_fa.nii.gz')).flatten()
        
        for i, val in enumerate(mask):
            if not np.isnan(val):
                base_arr[i] = cat
    
    for i, val in enumerate(base_arr):
        if val == 0:
            base_arr[i] = np.nan

    
    voxel_num = np.sum(~np.isnan(base_arr))
    base_arr = base_arr.reshape(fa.shape)
    nif_fa = to_mni(base_arr)

    nib.save(nif_fa, 
         os.path.join(save_path, 'fa_screened',
                    f'{cat}_{voxel_num}_fa.nii.gz'))
    end = time.time()
    print('--------------------------------------------')
    print(f'{cat} finished, time used: {end - start:.2f} seconds, ')
    total_num -= 1
    print(f'{total_num} left')

#### Construct New Data Frame

In [ ]:
data_path = os.path.join(save_path, 
                    'fa_screened')
files = os.listdir(data_path)

In [ ]:
ref_df = pd.read_csv(os.path.join(root_path, 'cluster_table_small_version',
                                'fa_attributes.csv'))

In [ ]:
df_dic_new = {}
for file in files:
    cat, num = int(file.split('_')[0]), int(file.split('_')[1])
    df_dic_new[cat] = {}

    mask = image.get_data(os.path.join(save_path, 'fa_screened',
                        file))
    mask = mask == cat
    model_shape = mask.shape
    mask = mask.flatten()
    mask_with_cortex = mask * cortex_mask
    fa_arr = fa.flatten()

    # get beta values specific to label
    arr_to_max = []
    for i, val in enumerate(mask_with_cortex):
        if val:
            arr_to_max.append(fa_arr[i])
        else:
            arr_to_max.append(np.nan)

    arr_to_max = np.array(arr_to_max).reshape(model_shape)
    arr_to_max = np.nan_to_num(arr_to_max)

    # get number of voxels
    df_dic_new[cat]['number_of_voxels'] = num 

    # get name of cluster 
    name_df = ref_df[ref_df['HO_categor_by_peak'] == cat]
    names = name_df['names_by_peak'].iloc[0]
    df_dic_new[cat]['cluster_name'] = names 

    # get mni coordinates
    x, y, z = np.unravel_index(np.argmax(arr_to_max), arr_to_max.shape)
    df_dic_new[cat]['mni_coordinates'] = image.coord_transform(x, y, z, 
                                                                niimg.affine)

    # get intensity 
    df_dic_new[cat]['z-score'] = (np.mean(fa_arr[mask]), np.std(fa_arr[mask]))

In [ ]:
df_fa_new = pd.DataFrame(df_dic_new).transpose()

df_fa_new.to_csv(os.path.join(root_path,
                        'cluster_table_small_version',
                        'fa_integrated.csv'))

## SA

### Watershed Parcellation 

In [ ]:
mask = np.isnan(sa)
sa = abs(sa)

data = sa
thresh = 0
mask = data > thresh 
sigma = 0.1
data = ndimage.gaussian_filter(data, sigma)
seeds = is_local_maximum(data)
seeds[~mask] = 0
se = ndimage.generate_binary_structure(3, 3)
markers = ndimage.label(seeds, se)[0]
seg_input = mask

parcel_map = segmentation.watershed(seg_input, markers, mask=mask) 

In [ ]:
selected_loi = find_loi(parcel_map, mask, 10) # threshold changed from 50 to 10

In [ ]:
parcel_map = parcel_map.astype('float') 

### Generate nii file for each label

In [ ]:
for label, num in selected_loi: 
    if label == 0: 
        continue
    mask_l = parcel_map.flatten() == label

    mni_vec = mask_cat(mask_l, parcel_map.flatten())
    nif_sa = to_mni(mni_vec)

    nib.save(nif_sa, 
         os.path.join(save_path, 'sa',
                    f'{label}_{num}_sa.nii.gz'))

In [ ]:
# gen selectec mask 
mni_vec = []
p_map = parcel_map.flatten()
selected_cat = [i for i, j in selected_loi]

for val in p_map:
    if val in selected_cat:
        mni_vec.append(val)
    else:
        mni_vec.append(np.nan)

        
nif_fa = to_mni(np.array(mni_vec))
nib.save(nif_fa, 
         os.path.join(save_path, 'sa',
                    f'full_sa.nii.gz'))

### Cluster Evaluation 

In [ ]:
table_dic = {}

for label, num in selected_loi:
    if label == 0:
        continue
    table_dic[label] = {}
    mask = image.get_data(os.path.join(save_path, 'sa',
                        f'{label}_{num}_sa.nii.gz'))
    mask = mask == label
    model_shape = mask.shape
    mask = mask.flatten()
    mask_with_cortex = mask * cortex_mask
    sa_arr = sa.flatten()

    # get beta values specific to label
    arr_to_max = []
    for i, val in enumerate(mask_with_cortex):
        if val:
            arr_to_max.append(fa_arr[i])
        else:
            arr_to_max.append(np.nan)

    arr_to_max = np.array(arr_to_max).reshape(model_shape)
    arr_to_max = np.nan_to_num(arr_to_max)

    # get cluster size
    table_dic[label]['voxel_size'] = num

    # get matrix coordinates
    x, y, z = np.unravel_index(np.argmax(arr_to_max), arr_to_max.shape)
    table_dic[label]['voxel_location'] = (x, y, z)

    # get mni coordinates
    table_dic[label]['mni_coordinates'] = image.coord_transform(x, y, z, 
                                                            niimg.affine)
    # get the right hemisphere or left hemisphere   
    hem = 'left ' if table_dic[label]['mni_coordinates'][0] < 0 else 'right '

    # get the label of H-O atlas 
    table_dic[label]['HO_categor_by_peak'] = ho_map[x, y, z]

    # get name of the cluster 
    names = mapper[table_dic[label]['HO_categor_by_peak']]
    table_dic[label]['names_by_peak'] = hem + names

    # sanity check, if the peak is in the left hemisphere
    if hem == 'left ':
        table_dic[label]['HO_categor_by_peak'] = -1 * ho_map[x, y, z]

    # get cluster name by covered region 
    label_count_arr = ho_map[mask.reshape(model_shape)]
    count_dic = Counter(label_count_arr)
    total = sum(count_dic.values())

    cat_percentile = {}
    for k in count_dic.keys():
        cat_percentile[k] = count_dic[k] / total 
    cat_percentile = sorted(cat_percentile.items(), key=lambda x: -x[1])

    table_dic[label]['percentile_distribution'] = cat_percentile
    table_dic[label]['HO_categor_by_area'] = cat_percentile[0][0]
    table_dic[label]['names_by_area'] = mapper[cat_percentile[0][0]]

    # get intensity 
    table_dic[label]['z-score'] = (np.mean(sa_arr[mask]), np.std(sa_arr[mask]))

In [ ]:
df_sa = pd.DataFrame(table_dic).transpose()

df_sa.to_csv(os.path.join(root_path,
                        'cluster_table_small_version',
                        'sa_attributes.csv'))

#### Modification of Clusters
1. Read the modified cluster csv file that is screened manually
2. the standard of cluster label is 'HO_categor_by_peak'
3. merge clusters with the same label to the finall nii file 

In [ ]:
df_sa = pd.read_csv(os.path.join(root_path, 'cluster_table_small_version',
                                'sa_attributes.csv'))

In [ ]:
cats = np.unique(df_sa['HO_categor_by_peak'])
sa_arr = sa.flatten()

for cat in cats:
    if cat == 0:
        continue    
    base_arr = np.zeros((sa_arr.size))
    df_temp = df_sa[df_sa['HO_categor_by_peak'] == cat ]
    for _, row in df_temp.iterrows():
        label = row['Unnamed: 0']
        num = row['voxel_size']
        mask = image.get_data(os.path.join(save_path, 'sa',
                        f'{label}_{num}_sa.nii.gz')).flatten()
        
        for i, val in enumerate(mask):
            if not np.isnan(val):
                base_arr[i] = cat
    
    for i, val in enumerate(base_arr):
        if val == 0:
            base_arr[i] = np.nan

    
    voxel_num = np.sum(~np.isnan(base_arr))
    base_arr = base_arr.reshape(sa.shape)
    nif_sa = to_mni(base_arr)

    nib.save(nif_sa, 
         os.path.join(save_path, 'sa_screened',
                    f'{cat}_{voxel_num}_sa.nii.gz'))

#### Construct New Data Frame

In [ ]:
data_path = os.path.join(save_path, 
                    'sa_screened')
files = os.listdir(data_path)

In [ ]:
ref_df = pd.read_csv(os.path.join(root_path, 'cluster_table_small_version',
                                'sa_attributes.csv'))

In [ ]:
df_dic_new = {}
for file in files:
    if file == '.DS_Store':
        continue
    cat, num = int(file.split('_')[0]), int(file.split('_')[1])
    df_dic_new[cat] = {}

    mask = image.get_data(os.path.join(save_path, 'sa_screened',
                        file))
    mask = mask == cat
    model_shape = mask.shape
    mask = mask.flatten()
    mask_with_cortex = mask * cortex_mask
    sa_arr = sa.flatten()

    # get beta values specific to label
    arr_to_max = []
    for i, val in enumerate(mask_with_cortex):
        if val:
            arr_to_max.append(sa_arr[i])
        else:
            arr_to_max.append(np.nan)

    arr_to_max = np.array(arr_to_max).reshape(model_shape)
    arr_to_max = np.nan_to_num(arr_to_max)

    # get number of voxels
    df_dic_new[cat]['number_of_voxels'] = num 

    # get name of cluster 
    name_df = ref_df[ref_df['HO_categor_by_peak'] == cat]
    names = name_df['names_by_peak'].iloc[0]
    df_dic_new[cat]['cluster_name'] = names 

    # get mni coordinates
    x, y, z = np.unravel_index(np.argmax(arr_to_max), arr_to_max.shape)
    df_dic_new[cat]['mni_coordinates'] = image.coord_transform(x, y, z, 
                                                                niimg.affine)

    # get intensity 
    df_dic_new[cat]['z-score'] = (np.mean(sa_arr[mask]), np.std(sa_arr[mask]))

In [ ]:
df_sa = pd.DataFrame(df_dic_new).transpose()

df_sa.to_csv(os.path.join(root_path,
                        'cluster_table_small_version',
                        'sa_integrated.csv'))